In [1]:
import pandas as pd
import os

### 1. 替换group和object

In [13]:
path = "data/normal.csv"
alter_object_path = "other/new_mapping_1.csv"
alter_group_path = "other/new_type.csv"

df = pd.read_csv(path)
alter_group_data = pd.read_csv(alter_group_path)
alter_object_data = pd.read_csv(alter_object_path)
print(len(list(df['sample'])))

alter_object_dict = {(row['object'], row['group']): row['mapping'] for idx, row in alter_object_data.iterrows()}
alter_group_dict = {(row['text'], row['type']): row['new_type'] for idx, row in alter_group_data.iterrows()}

243


In [14]:
mapping_obj = []
alter_groups = []

for idx, row in df.iterrows():
    objs = row['object'].split(", ")
    groups = row['group'].split(", ")
    alter_obj, alter_group = [], []
    
    for obj, group in zip(objs, groups):
        obj, group = obj.strip(), group.strip()
        try:
            tmp_obj = alter_object_dict[(obj, group)]
        except:
            print((obj, group), "未找到对应的替换名称")
        try:
            if group not in ['group1', 'group2', 'group3', 'group4']:
                tmp_group = alter_group_dict[(obj, group)]
            else:
                tmp_group = group
        except:
            print((obj, group), "未找到对应的可替换Group")
        alter_obj.append(tmp_obj)
        alter_group.append(tmp_group)
    mapping_obj.append(alter_obj)
    alter_groups.append(alter_group)

In [15]:
df.insert(4, 'mapping_object', mapping_obj)
df.insert(5, 'alter_group', alter_groups)

In [16]:
# example
for idx, row in df.iterrows():
    objs = row['object'].split(", ")
    groups = row['group'].split(", ")
    print(objs)
    print(mapping_obj[idx])
    print()
    print(groups)
    print(alter_groups[idx])
    print("\n\n")
    if idx > 20:
        break

['mother', 'son', 'daughter', 'son', 'the cookie jar', 'the cupboard door', 'he', 'cookies', 'jar', 'he', 'stool', 'girl', 'her hand', 'cookie', 'she', 'her finger', 'her lips', 'him', 'mother', 'she', 'dish', 'water', 'sink', 'she', 'curtains', 'garage', 'shrubbery', 'path', 'tree', 'plate', 'sugar bowl set', "boy's shoes", 'shoes', 'socks', 'he', 'shorts', 'shirt', 'sister', 'mini dress mini skirt', 'sandals', 'curtains', 'windows', 'cupboards']
['mother', 'boy', 'girl', 'boy', 'jar', 'cabinet', 'boy', 'cookie', 'jar', 'boy', 'stool', 'girl', 'girl', 'cookie', 'girl', 'girl', 'girl', 'boy', 'mother', 'mother', 'dish', 'water', 'sink', 'mother', 'curtain', 'garage', 'plant', 'path', 'plant', 'dish', 'bowl', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'girl', 'girl', 'girl', 'curtain', 'window', 'cabinet']

['group2', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1-girl-hand', 'group1', 'group1', 'group1-girl-hand', 'g

### 2. 滤去重复的group

In [17]:
def filter_consecutive_duplicates(input_list):
    filtered_list = []
    prev_element = None

    for element in input_list:
        if element != prev_element:
            filtered_list.append(element)
            prev_element = element

    return filtered_list

In [18]:
final_groups = []
count = 0
for alter_group in alter_groups:
    final_group = filter_consecutive_duplicates(alter_group)
    if len(final_group) != len(set(alter_group)):
        print(alter_group)
        print(set(alter_group))
        print(final_group)
        print()
        count += 1
    final_groups.append(final_group)

['group2', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group2', 'group2', 'group2', 'group2', 'group2', 'group2', 'group4', 'group4', 'group4', 'group4', 'group4', 'group3', 'group3', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group4', 'group4', 'group2']
{'group1', 'group2', 'group3', 'group4'}
['group2', 'group1', 'group2', 'group4', 'group3', 'group1', 'group4', 'group2']

['group1', 'group1', 'group1', 'group1', 'group1', 'group1', 'group2', 'group2', 'group2', 'group2', 'group2', 'group2', 'group4', 'group2', 'group4', 'group4', 'group3', 'group3', 'group3', 'group1']
{'group1', 'group2', 'group3', 'group4'}
['group1', 'group2', 'group4', 'group2', 'group4', 'group3', 'group1']

['group1', 'group1', 'group1', 'group1', 'group1', 'group2', 'group2', 'group2', 'group2', 'group2', 'group2', 'group2', 'group2', '

In [19]:
df.insert(6, 'final_group', final_groups)

In [20]:
df.to_csv(path, index=False)

### 3. 统计sample的词频， 出现在一个sample记为一次（最大词频为样本数）

In [40]:
import pandas as pd
import json
path = "other/new_mapping_1.csv"
df = pd.read_csv(path)

mapping_dict = dict(zip(df['object'], df['mapping']))

# object_count = {x :0 for x in set(df['mapping'])}

In [41]:
object_set = set(df['object'])
object_count ={
    'object': list(object_set),
    'normal_count': [0] * len(object_set),
    'abnormal_count': [0] * len(object_set),
    "count": [0] * len(object_set),              
}
def read_raw_file(file_path):
    return json.load(open(file_path, 'r'))

def _extract_text_one(file_path):
    if "abnormal" in file_path:
        symbol = "abnormal_count"
    else:
        symbol = "normal_count"
        
    data = read_raw_file(file_path)
    print(file_path)
    count = 0
    for i in range(len(data)):
        annotations = data[i]['annotations']
        annotations = annotations[
            len(annotations) - 1]  # select the newest annotations when multiple annotations exist
        
        # 收集所有object
        keys = []
        for single_entity in annotations['result']:
            count += 1
            text = single_entity['value']['text'].strip()
            
            key = mapping_dict[text]  # 统计mapping后的
            key = text  # 统计原始obj
            keys.append(key)
        
        # for key in list(set(keys)):
        for key in list(keys): # 词频
            pos = object_count['object'].index(key)
            
            object_count[symbol][pos] += 1
            object_count['count'][pos] += 1

        all_count = 0
        for d in object_count[symbol]:
            all_count += d
    print(f"{file_path}共{count}个object,共统计了{all_count}个")

def extract_text_all(file_folder):
    for file_path in os.listdir(file_folder):
        if not file_path.endswith('.json'):
            continue
        _extract_text_one(os.path.join(file_folder, file_path))    

In [42]:
extract_text_all("data/final/")

data/final/abnormal.json
data/final/abnormal.json共4665个object,共统计了4665个
data/final/normal.json
data/final/normal.json共4832个object,共统计了4832个


In [43]:
df = pd.DataFrame(object_count)
df.to_csv("freq/obj_freq_2.csv", index=False)

### 4. Feature 1: 350-549 high, 100-350 mid-high,10-100 mid, <10 low freq 1的模式，numerical feature

In [1]:
import pandas as pd
from collections import Counter
freq_data = pd.read_csv("freq/mapping_freq_1.csv")

freq_data_dict = {
    'high': freq_data[(freq_data['count'] >= 350) & (freq_data['count'] <= 549)]['object'].tolist(),
    'mid-high': freq_data[(freq_data['count'] >= 100) & (freq_data['count'] < 350)]['object'].tolist(),
    'mid': freq_data[(freq_data['count'] >= 10) & (freq_data['count'] < 100)]['object'].tolist(),
    'low': freq_data[freq_data['count'] < 10]['object'].tolist()
}

In [2]:
freq_data_dict

{'high': ['mother', 'jar', 'dish', 'cookie', 'water', 'stool', 'boy', 'sink'],
 'mid-high': ['window', 'cabinet', 'cup', 'plant', 'girl', 'curtain', 'floor'],
 'mid': ['path', 'faucet', 'towel', 'kids', 'garage', 'yard', 'lid', 'house'],
 'low': ['board',
  'table',
  'wall',
  'corner',
  'mop',
  'handle',
  'bowl',
  'bird',
  'button']}

In [57]:
path = "data/normal.csv"
data = pd.read_csv(path, usecols=['sample', 'label', 'mapping_object', 'final_group'])

In [58]:
from typing import List
import numpy as np
from sklearn.metrics import ndcg_score

def orderness(group_order: List[int]):
    if len(group_order) == 1:
        return 1.0
    true_order_ascending = np.array([sorted(group_order, reverse=False)])
    true_order_descending = np.array([sorted(group_order, reverse=True)])
    group_order = np.array([group_order])
    
#     print(true_order_ascending, group_order)
#     print(true_order_descending, group_order)
#     print("\n\n\n")
    return max(ndcg_score(true_order_ascending, group_order), ndcg_score(true_order_descending, group_order))

In [59]:
Feature_1 = {'high': [], 'mid-high': [], 'mid': [], 'low': [],}
Feature_2_1 = {'high': [], 'mid-high': [], 'mid': [], 'low': [],}
Feature_2_2 = {x: [] for x in freq_data_dict['high']}
Feature_3 = []
Feature_4 = []

for idx, row in data.iterrows():  # 遍历每一个样本
    mapping_objs = eval(row['mapping_object'])
    final_group = eval(row['final_group'])
    mapping_obj_count = Counter(mapping_objs)
    # 找每一种特征
    for freq, freq_objs in freq_data_dict.items():
        # Feature_1
        ratio = len([x for x in freq_objs if x in mapping_objs]) / len((freq_objs))
        ratio = round(ratio, 3)
#         print(freq_objs)
#         print([x for x in freq_objs if x in mapping_objs])
#         print(freq, ratio)
        Feature_1[freq].append(ratio)
    
        # Feature_2_1
        duplicated = [x for x in freq_objs if mapping_obj_count[x] > 1]
        ratio = len(duplicated) / len((freq_objs))
        ratio = round(ratio, 3)
#         print(duplicated, len(freq_objs), ratio)
        Feature_2_1[freq].append(ratio)
        
    # Feature_2_2
    for obj in Feature_2_2.keys():
        count = mapping_obj_count[obj]
        Feature_2_2[obj].append(count)
    
    # Feature_3
    group_count = len(set(final_group))
    ratio = group_count / 4
    ratio = round(ratio, 3)
    Feature_3.append(ratio)
    
    # Feature_4
    group_idx = [int(x[-1]) for x in final_group]
    Feature_4.append(round(orderness(group_idx), 3))

In [60]:
new_df = pd.DataFrame(data, columns=['sample', 'label'])

In [61]:
idx = 2
for key, value in Feature_1.items():
    new_df.insert(idx, f"Feature_1_{key}", value)
    idx += 1

In [62]:
for key, value in Feature_2_1.items():
    new_df.insert(idx, f"Feature_2_{key}", value)
    idx += 1

In [63]:
for key, value in Feature_2_2.items():
    new_df.insert(idx, f"Feature_2_{key}", value)
    idx += 1

In [64]:
new_df.insert(idx, f"Feature_3", Feature_3)
idx += 1
new_df.insert(idx, f"Feature_4", Feature_4)

In [65]:
new_df.to_csv("feature/normal.csv", index=False)

### embedding 相似度

In [58]:
from nltk.tokenize import sent_tokenize
import json
import pandas as pd
import os

os.environ['CUDA_VISIBLE_DEVICES'] = "2"
from sentence_transformers import SentenceTransformer, util
import math
model = SentenceTransformer('./all-MiniLM-L6-v2/')
path = "data/final/normal.json"


In [59]:
def read_raw_file(file_path):
    return json.load(open(file_path, 'r'))

In [60]:
data = read_raw_file(path)
score_dict = {}
for i in range(len(data)):
    para = data[i]['data']['text'].replace("\t", ' ')
    s = data[i]['file_upload']
    sample = s[s.find('-') + 1: -8]
    # print(para.replace("\t", ' '))
    
    sentences = [x.strip() for x in para.split(".")]
    if not sentences[-1]:
        sentences = sentences[:-1]

    embeddings = model.encode(sentences)

    #Compute cosine similarity between all pairs
    cos_sim = util.cos_sim(embeddings, embeddings)
    
    #Add all pairs to a list with their cosine similarity score
    all_sentence_combinations = []
    for i in range(len(cos_sim)-1):
        for j in range(i+1, len(cos_sim)):
            all_sentence_combinations.append([cos_sim[i][j], i, j])
            
    scores = []
    for score, i, j in all_sentence_combinations:
        # print("{} \t {} \t {:.4f}".format(sentences[i], sentences[j], cos_sim[i][j]))
        scores.append(score * math.log10(j - i))
        # scores.append(score)
        
    score = sum(scores) / len(scores)

    score_dict[sample] = round(float(score), 4)

df = pd.read_csv("./feature/normal.csv")
scores = [score_dict[x] for x in list(df['sample'])]
df.insert(21, 'log_embed_ds', scores)
df.to_csv("./feature/normal.csv", index=False)

### 添加句子、词 个数

In [11]:
import json
import pandas as pd
import os
def read_raw_file(file_path):
    return json.load(open(file_path, 'r'))

In [13]:
path = "data/final/abnormal.json"
data = read_raw_file(path)
sen_dict = {}
word_dict = {}
for i in range(len(data)):
    para = data[i]['data']['text'].replace("\t", ' ')
    s = data[i]['file_upload']
    sample = s[s.find('-') + 1: -8]
    # print(para.replace("\t", ' '))
    
    sentences = [x.strip() for x in para.split(".")]
    if not sentences[-1]:
        sentences = sentences[:-1]
    
    word_count = 0
    for sentence in sentences:
        words = [x for x in sentence.split(' ') if x.strip()]
        # print(sentence, words)
        word_count += len(words)

    sen_dict[sample] = len(sentences)
    word_dict[sample] = word_count
    # print(sample, sen_dict[sample], word_dict[sample])

    # print("\n\n")
    

df = pd.read_csv("./feature/merged_abnormal.csv")
sentences_num = [sen_dict[x] for x in list(df['sample'])]
words_num = [word_dict[x] for x in list(df['sample'])]
try:
    df.insert(22, 'sentence_num', sentences_num)
    df.insert(23, 'words_num', words_num)
except:
    df['sentence_num'] = sentences_num
    df['words_num'] = words_num
df.to_csv("./feature/merged_abnormal.csv", index=False)

### 添加Clip Image-Text score

In [2]:
import pandas as pd
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import os
from collections import defaultdict

datapath = "./all_data.csv"
df = pd.read_csv(datapath)
image_dict = {
    'group1': "./images/res_group1.png",
    'group2': "./images/res_group2.png",
    'group3': "./images/res_group3.png",
    'group4': "./images/res_group4.png"
}

In [3]:
model = CLIPModel.from_pretrained("/hdd/wmh/clip").cuda()
processor = CLIPProcessor.from_pretrained("/hdd/wmh/clip")

In [4]:
samples = []
group1_scores, group2_scores, group3_scores, group4_scores = [], [], [], []
sample_scores = []

In [5]:
groups_df = df.groupby('sample')

In [6]:
for name, group in groups_df:
    sample = name
    # print(group)
    s_g = {}
    for _, row in group.iterrows():
        if isinstance(row['groups'], float):
            s_g[row['sentence']] = []
        else:
            s_g[row['sentence']] = list(set(row['groups'].split(', ')))


    g_s = defaultdict(list)
    for k, v in s_g.items():
        for x in v:
            g_s[x.strip()].append(k)

    tmp_scores = {
        'group1': [],
        'group2': [],
        'group3': [],
        'group4': []
    }
    # 对每一个group进行clip计算
    for k, v in g_s.items():
        imagepath = image_dict[k]
        image = Image.open(imagepath)
        inputs = processor(text=v, images=image, return_tensors="pt", padding=True)
        outputs = model(**inputs.to('cuda'))
        logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
        # print(logits_per_image)
        tmp_scores[k] += logits_per_image.tolist()[0]

    samples.append(sample)

    for k, v in tmp_scores.items():
        if not v:
            tmp_scores[k] = [0.0]
    
    group1_scores.append(sum(tmp_scores['group1']) / len(tmp_scores['group1']))
    group2_scores.append(sum(tmp_scores['group2']) / len(tmp_scores['group2']))
    group3_scores.append(sum(tmp_scores['group3']) / len(tmp_scores['group3']))
    group4_scores.append(sum(tmp_scores['group4']) / len(tmp_scores['group4']))

In [46]:
data = pd.DataFrame(
    {
        'sample': samples,
        'group1_score': [round(x, 4) for x in group1_scores],
        'group2_score': [round(x, 4) for x in group2_scores],
        'group3_score': [round(x, 4) for x in group3_scores],
        'group4_score': [round(x, 4) for x in group4_scores],
    }
)

In [47]:
data.to_csv("feature/clip_score.csv", index=False)

In [112]:
# 提取得分
import pandas as pd
path = "feature/merged_abnormal.csv"
ref_path = "feature/clip_score.csv"
df = pd.read_csv(path)
ref_df = pd.read_csv(ref_path)

In [113]:
sub_ref_df = ref_df[ref_df['sample'].isin(df['sample'])].reset_index(drop=True)

In [114]:
# 比较是否相同
list(df['sample']) == list(sub_ref_df['sample'])

True

In [115]:
df.shape, sub_ref_df.shape

((306, 57), (306, 5))

In [116]:
df = pd.concat([df, sub_ref_df.drop('sample', axis=1)], axis=1)

In [27]:
df.to_csv(path, index=False)

In [38]:
import pandas as pd
path = "feature/merged_normal.csv"
df = pd.read_csv(path)

In [39]:
avg_clip_score = []
for _, row in df.iterrows():
    s1 = row['group1_score']
    s2 = row['group2_score']
    s3 = row['group3_score']
    s4 = row['group4_score']
    avg_ = sum([s1, s2, s3, s4]) / 4
    avg_clip_score.append(round(avg_, 4))

In [40]:
df

,sample,label,Feature_1_high,Feature_1_mid-high,Feature_1_mid,Feature_1_low,Feature_2_high,Feature_2_mid-high,Feature_2_mid,Feature_2_low,...,Feature_2_sink,Feature_3,Feature_4,log_embed_ds,sentence_num,words_num,group1_score,group2_score,group3_score,group4_score
0,002-0,0,0.875,0.571,0.125,0.000,0.500,0.000,0.000,0.0,...,0,0.75,0.859,0.0763,21,140,25.3855,26.0828,0.0000,22.3558
1,002-1,0,1.000,0.714,0.000,0.000,0.625,0.571,0.000,0.0,...,1,0.75,0.816,0.1252,20,112,23.1376,24.0490,0.0000,22.6075
2,002-2,0,0.875,0.571,0.125,0.000,0.375,0.143,0.125,0.0,...,1,0.75,0.860,0.0663,18,168,24.8766,25.4910,0.0000,23.2258
3,002-3,0,0.875,0.286,0.250,0.000,0.375,0.143,0.125,0.0,...,1,0.75,0.871,0.0956,14,188,25.2477,24.7975,0.0000,21.3510
4,006-2,0,1.000,0.429,0.000,0.000,0.500,0.143,0.000,0.0,...,1,1.00,0.889,0.1074,16,91,24.4101,24.1688,22.6000,23.5057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,686-0,0,0.875,0.857,0.500,0.111,0.625,0.571,0.000,0.0,...,1,1.00,0.902,0.1077,14,188,24.2112,25.2519,23.5352,22.6138
239,688-0,0,0.750,0.286,0.000,0.000,0.125,0.000,0.000,0.0,...,0,0.75,0.980,0.0852,14,104,25.1225,23.3745,0.0000,24.0504
240,691-0,0,0.750,0.857,0.375,0.000,0.375,0.143,0.000,0.0,...,1,1.00,0.869,0.1274,22,171,21.8147,24.3520,20.7438,21.7715
241,709-0,0,0.875,0.286,0.000,0.000,0.250,0.000,0.000,0.0,...,1,0.75,0.865,0.1119,14,96,23.9415,22.6698,22.2225,0.0000


In [41]:
df.insert(22, 'avg_clip_score', avg_clip_score)

In [42]:
df.to_csv(path, index=False)

In [27]:
# change word_num feature to words / sentences
import pandas as pd
path = "./feature/merged_normal.csv"
data = pd.read_csv(path)

In [33]:
data.columns

Index(['sample', 'label', 'Feature_1_high', 'Feature_1_mid-high',
       'Feature_1_mid', 'Feature_1_low', 'Feature_2_high',
       'Feature_2_mid-high', 'Feature_2_mid', 'Feature_2_low',
       'Feature_2_mother', 'Feature_2_jar', 'Feature_2_dish',
       'Feature_2_cookie', 'Feature_2_water', 'Feature_2_stool',
       'Feature_2_boy', 'Feature_2_sink', 'Feature_3', 'Feature_4',
       'log_embed_ds', 'sentence_num', 'words_num', 'word_per_sentence',
       'group1_score', 'group2_score', 'group3_score', 'group4_score',
       'avg_clip_score', 'Linguistic', 'function', 'pronoun', 'ppron', 'i',
       'we', 'you', 'shehe', 'they', 'ipron', 'det', 'article', 'number',
       'prep', 'auxverb', 'adverb', 'conj', 'negate', 'verb', 'adj',
       'quantity'],
      dtype='object')

In [29]:
words_per_sentence = []
for _, row in data.iterrows():
    print(row['sentence_num'], row['words_num'])
    words_per_sentence.append(round(row['words_num'] / row['sentence_num'], 4))

21 140
20 112
18 168
14 188
16 91
14 143
17 188
16 102
15 69
15 95
19 110
30 220
21 169
34 275
27 145
22 158
33 255
16 104
16 177
11 62
15 56
11 66
19 104
8 73
11 85
24 151
14 163
19 110
11 63
15 114
16 120
19 147
21 164
11 78
11 83
10 60
23 130
20 111
18 113
16 88
4 32
15 119
21 93
15 99
9 73
10 72
12 89
10 79
10 77
12 94
11 90
9 137
22 96
22 81
25 99
23 133
16 76
20 121
8 76
16 121
35 268
19 127
17 113
20 134
21 172
10 69
19 98
7 48
10 59
27 97
12 77
22 141
20 111
15 163
16 97
23 177
13 82
9 45
22 119
14 120
20 99
15 72
15 81
20 113
12 81
18 103
12 76
15 57
9 64
14 75
10 69
12 111
13 53
13 102
11 117
26 273
28 152
24 200
22 191
23 206
17 98
24 128
25 231
36 368
45 485
15 125
7 44
10 50
8 80
17 96
16 83
26 159
15 91
12 73
15 137
19 133
13 132
18 116
11 52
6 43
25 166
12 68
14 79
8 63
12 99
24 114
17 122
14 75
14 77
10 73
18 51
20 191
19 118
16 98
16 122
20 160
10 67
21 119
9 76
19 164
22 102
16 78
13 73
8 52
20 157
15 99
11 98
13 77
23 214
10 59
9 53
21 153
25 177
16 165
23 146
10 76


In [30]:
data.insert(23, 'word_per_sentence', words_per_sentence)

In [31]:
order = ['sample', 'label', 'Feature_1_high', 'Feature_1_mid-high',
        'Feature_1_mid', 'Feature_1_low', 'Feature_2_high',
        'Feature_2_mid-high', 'Feature_2_mid', 'Feature_2_low',
        'Feature_2_mother', 'Feature_2_jar', 'Feature_2_dish',
        'Feature_2_cookie', 'Feature_2_water', 'Feature_2_stool',
        'Feature_2_boy', 'Feature_2_sink', 'Feature_3',
        'Feature_4', 'log_embed_ds', 'sentence_num',
        'words_num', 'word_per_sentence', 'group1_score', 'group2_score', 'group3_score',
        'group4_score', 'avg_clip_score', 'Linguistic', 'function', 'pronoun', 'ppron', 'i', 'we',
        'you', 'shehe', 'they', 'ipron', 'det', 'article', 'number', 'prep',
        'auxverb', 'adverb', 'conj', 'negate', 'verb', 'adj', 'quantity']

In [32]:
data = data[order]
data.to_csv(path, index_label=False)

In [34]:
# 提取feature2 词频
import pandas as pd
from collections import Counter
freq_data = pd.read_csv("freq/mapping_freq_1.csv")

freq_data_dict = {
    'high': freq_data[(freq_data['count'] >= 350) & (freq_data['count'] <= 549)]['object'].tolist(),
    'mid-high': freq_data[(freq_data['count'] >= 100) & (freq_data['count'] < 350)]['object'].tolist(),
    'mid': freq_data[(freq_data['count'] >= 10) & (freq_data['count'] < 100)]['object'].tolist(),
    'low': freq_data[freq_data['count'] < 10]['object'].tolist()
}

In [48]:
path1 = "./data/normal.csv"
path2 = "./feature/merged_normal.csv"
data = pd.read_csv(path1, usecols=['sample', 'label', 'mapping_object', 'final_group'])
df = pd.read_csv(path2)
data.sort_values(by='sample', inplace=True)

In [49]:
list(df['sample']) == list(data['sample'])

True

In [50]:
need_count = freq_data_dict['mid-high']

In [51]:
Feature_2_2 = {x: [] for x in need_count}

for idx, row in data.iterrows():  # 遍历每一个样本
    mapping_objs = eval(row['mapping_object'])
    final_group = eval(row['final_group'])
    mapping_obj_count = Counter(mapping_objs)
    # 找每一种特征
        
    # Feature_2_2
    for obj in Feature_2_2.keys():
        count = mapping_obj_count[obj]
        Feature_2_2[obj].append(count)

In [52]:
idx = 18
for key, value in Feature_2_2.items():
    df.insert(idx, f"Feature_2_{key}", value)
    idx += 1

In [53]:
df.to_csv(path2, index=False)

In [54]:
a = "./feature/merged_normal.csv"
b = "./feature/merged_abnormal.csv"
c = "./feature/merged_normal_0.75.csv"

In [55]:
df1 = pd.read_csv(a)
df2 = pd.read_csv(b)
df3 = pd.read_csv(c)

In [57]:
df1.columns == df2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [56]:
len(df1.columns), len(df3.columns)

(57, 45)

In [60]:
[x for x in list(df1.columns) if x not in list(df3.columns)]

['Feature_1_mid',
 'Feature_2_mid',
 'Feature_2_low',
 'Feature_2_dish',
 'Feature_2_boy',
 'Feature_2_window',
 'Feature_2_cabinet',
 'Feature_2_cup',
 'Feature_2_plant',
 'Feature_2_girl',
 'Feature_2_curtain',
 'Feature_2_floor']

In [27]:
import pandas as pd
import os

In [28]:
src_path = "./feature/pos_tag.csv"
src_df = pd.read_csv(src_path)
cols = list(src_df.columns)
src_df = src_df[cols[1:]]
src_df = src_df.drop('label', axis=1)

In [29]:
user_id = [x[:5] for x in list(src_df['user_id'])]
src_df['user_id'] = user_id

In [30]:
path = "./feature/merged_abnormal.csv"
df = pd.read_csv(path)

In [31]:
need_user = list(df['sample'])

In [ ]:
src_df = src_df[src_df['user_id'].isin(need_user)]
src_df.sort_values(by='user_id', inplace=True)

src_df = src_df.reset_index(drop=True)

In [21]:
list(src_df['user_id']) == list(df['sample'])

True

In [22]:
src_df = src_df.drop('user_id', axis=1)

In [23]:
df.shape, src_df.shape

((306, 57), (306, 33))

In [24]:
df = pd.concat([df, src_df], axis=1)

In [25]:
df.shape

(306, 90)

In [26]:
src_df

,NN,EX,PDT,JJR,JJ,FW,IN,WRB,WDT,RB,...,VBZ,CC,PRP,PRP$,POS,",",VB,JJS,UH,VBN
0,18,1,0,0,4,0,11,0,1,9,...,10,8,7,0,0,0,4,0,0,0
1,26,3,1,1,5,0,17,0,0,8,...,15,8,10,0,0,0,7,0,0,0
2,39,1,0,0,10,0,18,1,0,21,...,12,17,8,0,0,2,12,0,0,0
3,15,0,0,0,3,0,3,0,0,6,...,7,0,3,0,1,0,4,0,0,0
4,20,1,1,0,1,0,3,0,0,5,...,4,1,5,0,0,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,25,0,0,0,5,0,10,0,1,4,...,5,3,2,2,2,1,3,0,0,0
302,21,0,0,0,2,0,12,0,0,11,...,6,4,10,0,0,0,6,0,0,0
303,29,2,0,0,5,0,10,0,2,3,...,10,10,8,2,0,0,6,0,0,0
304,22,3,0,0,4,0,14,0,1,9,...,8,19,19,2,1,0,11,0,0,2
